# データ型とデータ構造

In [1]:
import os

import polars as pl

## ネストされたデータ型
polarsでは、ネストされたデータ型は特別なクラスとして扱われる。  
ネストされたデータ型の要素にはプリミティブ型が使用でき、ネストされたデータ型には`Array / List / Struct`の3つがある。

array型

In [2]:
array_df = pl.DataFrame(
    [
        pl.Series('array_1', [[1, 3], [2, 5]]),
        pl.Series('array_2', [[1, 7, 3], [8, 1, 0]]),
    ],
    schema={
        'array_1': pl.Array(shape=2, inner=pl.Int64),
        'array_2': pl.Array(shape=3, inner=pl.Int64)
    }
)

array_df

array_1,array_2
"array[i64, 2]","array[i64, 3]"
"[1, 3]","[1, 7, 3]"
"[2, 5]","[8, 1, 0]"


Array型では、pythonのarray型と同様に要素の型が統一されている必要がある。  
しかし、DataFrameの全ての行で、ネストされたデータの長さが一致している必要はない（上記のように、各行/列でArray型の長さが異なっていてもOK）

（余談）  
numpyだと型を指定する際に`int64`みたいな扱い方をしたけれど、polarsでは`pl.Int64`というように大文字から始めるのか。Golangみたい

list型

In [3]:
list_df = pl.DataFrame(
    {
        'intege_lists': [[1, 2], [3, 4]],
        'float_lists': [[1.0, 2.0], [3.0, 4.0]],
    }
)

list_df

intege_lists,float_lists
list[i64],list[f64]
"[1, 2]","[1.0, 2.0]"
"[3, 4]","[3.0, 4.0]"


Struct型

In [4]:
rating_series = pl.Series(
    'ratings',
    [
        {'Movie': 'Cars', 'Theatre': 'NE', 'Avg_Raging': 4.5},
        {'Movie': 'Toy Story', 'Theatre': 'ME', 'Avg_Rating': 4.9}
    ],
)

rating_series

ratings
struct[3]
"{""Cars"",""NE"",4.5}"
"{""Toy Story"",""ME"",null}"


Struct型は複数の列を操作する時に使用する型。DataFrameの要素に複数の情報を含めたい場合に使用される。  
DataFrame in Series(or DataFrame)的なことが実現したいときに使われるのかな？つまり、3次元以上のDataFrameを扱うことができるイメージ？

## 欠損値

In [5]:
df = pl.DataFrame(
    {
        'value': [None, 2, 3, 4, None, None, 7, 8, 9, None],
    },
)

print(df)

shape: (10, 1)
┌───────┐
│ value │
│ ---   │
│ i64   │
╞═══════╡
│ null  │
│ 2     │
│ 3     │
│ 4     │
│ null  │
│ null  │
│ 7     │
│ 8     │
│ 9     │
│ null  │
└───────┘


### `null`を指定した数値で置換

In [6]:
print(
    df
    .with_columns(
        pl.col('value')
        .fill_null(-1)
        .alias('filled_with_lit')
    )
)

shape: (10, 2)
┌───────┬─────────────────┐
│ value ┆ filled_with_lit │
│ ---   ┆ ---             │
│ i64   ┆ i64             │
╞═══════╪═════════════════╡
│ null  ┆ -1              │
│ 2     ┆ 2               │
│ 3     ┆ 3               │
│ 4     ┆ 4               │
│ null  ┆ -1              │
│ null  ┆ -1              │
│ 7     ┆ 7               │
│ 8     ┆ 8               │
│ 9     ┆ 9               │
│ null  ┆ -1              │
└───────┴─────────────────┘


こうしてみると、確かにpandasみたいに`[`,`]`を乱用することなく記述できるのか。なるほど

(余談)  
polarsに限った話かは不明だが、改行のタイミングは`.~~()`のときなのか。筆者のクセかもしれないから断定するつもりはないけれど

### モジュールで用意されてる手法を使用する

In [7]:
print(
    df
    .with_columns(
        pl.col("value")
        .fill_null(strategy="forward")
        .alias("forward"),
        pl.col("value")
        .fill_null(strategy="backward")
        .alias("backward"),
        pl.col("value")
        .fill_null(strategy="min")
        .alias("min"),
        pl.col("value")
        .fill_null(strategy="max")
        .alias("max"),
        pl.col("value")
        .fill_null(strategy="mean")
        .alias("mean"),
        pl.col("value")
        .fill_null(strategy="zero")
        .alias("zero"),
        pl.col("value")
        .fill_null(strategy="one")
        .alias("one"),
    )
)

shape: (10, 8)
┌───────┬─────────┬──────────┬─────┬─────┬──────┬──────┬─────┐
│ value ┆ forward ┆ backward ┆ min ┆ max ┆ mean ┆ zero ┆ one │
│ ---   ┆ ---     ┆ ---      ┆ --- ┆ --- ┆ ---  ┆ ---  ┆ --- │
│ i64   ┆ i64     ┆ i64      ┆ i64 ┆ i64 ┆ i64  ┆ i64  ┆ i64 │
╞═══════╪═════════╪══════════╪═════╪═════╪══════╪══════╪═════╡
│ null  ┆ null    ┆ 2        ┆ 2   ┆ 9   ┆ 5    ┆ 0    ┆ 1   │
│ 2     ┆ 2       ┆ 2        ┆ 2   ┆ 2   ┆ 2    ┆ 2    ┆ 2   │
│ 3     ┆ 3       ┆ 3        ┆ 3   ┆ 3   ┆ 3    ┆ 3    ┆ 3   │
│ 4     ┆ 4       ┆ 4        ┆ 4   ┆ 4   ┆ 4    ┆ 4    ┆ 4   │
│ null  ┆ 4       ┆ 7        ┆ 2   ┆ 9   ┆ 5    ┆ 0    ┆ 1   │
│ null  ┆ 4       ┆ 7        ┆ 2   ┆ 9   ┆ 5    ┆ 0    ┆ 1   │
│ 7     ┆ 7       ┆ 7        ┆ 7   ┆ 7   ┆ 7    ┆ 7    ┆ 7   │
│ 8     ┆ 8       ┆ 8        ┆ 8   ┆ 8   ┆ 8    ┆ 8    ┆ 8   │
│ 9     ┆ 9       ┆ 9        ┆ 9   ┆ 9   ┆ 9    ┆ 9    ┆ 9   │
│ null  ┆ 9       ┆ null     ┆ 2   ┆ 9   ┆ 5    ┆ 0    ┆ 1   │
└───────┴─────────┴──────────┴─────┴────

`forward`は直前の行の数値を使用して、`backward`は直後の行の数値を使用する。  
気を付ける必要がある点は、`strategy`によって指定した手法で扱うデータ型がint型の場合は、計算結果がint型になるように収められてしまう。  
float型で扱いたい場合は、以下のように実行する

### 型の扱いを変えたいver  
pandasに近いやり方な気がする

In [8]:
print(
    df
    .with_columns(
        pl.col("value")
        .fill_null(pl.col("value").mean())
        .alias("expression_mean")
    )
)

shape: (10, 2)
┌───────┬─────────────────┐
│ value ┆ expression_mean │
│ ---   ┆ ---             │
│ i64   ┆ f64             │
╞═══════╪═════════════════╡
│ null  ┆ 5.5             │
│ 2     ┆ 2.0             │
│ 3     ┆ 3.0             │
│ 4     ┆ 4.0             │
│ null  ┆ 5.5             │
│ null  ┆ 5.5             │
│ 7     ┆ 7.0             │
│ 8     ┆ 8.0             │
│ 9     ┆ 9.0             │
│ null  ┆ 5.5             │
└───────┴─────────────────┘


### 線形補完
線形補完では前後に情報が必要であるため、DataFrameの先頭と末端がnullの場合はnullのままになってしまう

In [9]:
print(
    df.interpolate()
)

shape: (10, 1)
┌───────┐
│ value │
│ ---   │
│ f64   │
╞═══════╡
│ null  │
│ 2.0   │
│ 3.0   │
│ 4.0   │
│ 5.0   │
│ 6.0   │
│ 7.0   │
│ 8.0   │
│ 9.0   │
│ null  │
└───────┘


## Series / DataFrame / LazyFrame

In [10]:
string_df = pl.DataFrame({'id': ['10000', '20000', '30000']})
print(string_df)
print(f"Estimated size: {string_df.estimated_size('b')} bytes")

shape: (3, 1)
┌───────┐
│ id    │
│ ---   │
│ str   │
╞═══════╡
│ 10000 │
│ 20000 │
│ 30000 │
└───────┘
Estimated size: 15 bytes


上記の結果を踏まえて、メモリを節約する

In [11]:
int_df = string_df.select(pl.col('id').cast(pl.UInt16))
print(int_df)
print(f"Estimated size: {int_df.estimated_size('b')} bytes")

shape: (3, 1)
┌───────┐
│ id    │
│ ---   │
│ u16   │
╞═══════╡
│ 10000 │
│ 20000 │
│ 30000 │
└───────┘
Estimated size: 6 bytes


string型のときは1文字あたり1バイト使用していたが、16ビットの符号なし整数型を使うことで、1つの数字あたり2バイトで収まっている。  
こう考えてみると、適切な型を指定するだけでメリットって莫大だなぁ

In [12]:
df = pl.DataFrame(
    {
        'id': [10000, 20000, 30000],
        'value': [1.0, 2.0, 3.0],
        'value2': ['1', '2', '3'],
    }
)

df.cast(pl.UInt16)

id,value,value2
u16,u16,u16
10000,1,1
20000,2,2
30000,3,3


列を指定しなくても、DataFrame全体に対してcastすることが可能

In [13]:
df.cast({'id': pl.UInt16, 'value': pl.Float32, 'value2': pl.UInt8})

id,value,value2
u16,f32,u8
10000,1.0,1
20000,2.0,2
30000,3.0,3


一括で、別々のカラムの型をcastすることも可能。  
例えばデータの前処理で、フラグに相当するカラムとかは`UInt8/Int8`とかを使うことでメモリの節約ができるのか

In [14]:
df.cast({pl.Float64: pl.Float32, pl.String: pl.UInt8})

id,value,value2
i64,f32,u8
10000,1.0,1
20000,2.0,2
30000,3.0,3


この使い方は場合によってはめちゃくちゃ便利だけれど、やらかす可能性もありそう..